In [12]:
import os
import csv
import pandas as pd
import numpy as np
# rows in interactions file: 519293333

In [13]:
np.random.seed(42)

track_ids = np.arange(50813373) # last track id is 50813372, arange gives [a,b) so we add 1
np.random.shuffle(track_ids)
sampled_tracks = set(track_ids[:100000])

In [14]:
users_set = set()
tracks_set = set()
i = 0

with pd.read_csv('dataset/user_artist_track.tsv', sep='\t', names=['user_id', 'track_id', 'play_count'], chunksize=10 ** 6) as reader:
    for chunk in reader:
        users_set.update(chunk['user_id'].values)
        tracks_set.update(chunk['track_id'].values)
        
        i = i + len(chunk)
        print(i, end='\r')
    
print("\n")
print("\n")
print(len(users_set))
print(len(tracks_set))

519293333



120322
50813373


In [15]:
users = pd.read_csv('dataset/user_demographics.tsv', sep='\t', names=['user_id', 'country', 'age', 'gender', 'timestamp'])
users = users[['user_id','gender']]
users = users[users['gender'] != 'n']
users

,user_id,gender
1,1,m
2,2,m
4,4,m
5,5,m
7,7,m
...,...,...
120304,120304,f
120308,120308,m
120310,120310,f
120312,120312,m


In [16]:
user_ids = set(users['user_id'].values)

In [17]:
interactions_subset = pd.DataFrame(columns=['user_id', 'track_id', 'play_count']) # empty df
with pd.read_csv('dataset/user_artist_track.tsv', sep='\t', names=['user_id', 'track_id', 'play_count'], chunksize=10 ** 6) as reader:
    for chunk in reader:
        current = chunk[chunk['play_count'] > 1]
        current = current[current['user_id'].isin(user_ids)]
        current = current[current['track_id'].isin(sampled_tracks)]
        
        interactions_subset = pd.concat([interactions_subset, current])
        print(len(interactions_subset), end='\r')

In [18]:
def iterative_drop(interactions_data, user_threshold, track_threshold):
    while True:
        n = len(interactions_data)
        
        # filter users with enough interactions (have listened to enough different tracks)
        user_count = interactions_data['user_id'].value_counts()
        user_count = user_count[user_count >= user_threshold]
        interactions_data = interactions_data[interactions_data['user_id'].isin(set(user_count.index))]
        
        # filter tracks with enough interactions (have been listened to by enough different users)
        item_count = interactions_data['track_id'].value_counts()
        item_count = item_count[item_count >= track_threshold]
        interactions_data = interactions_data[interactions_data['track_id'].isin(set(item_count.index))]
        
        new_n = len(interactions_data)        
        if new_n == n:
            break
            
        n = new_n
    
    return interactions_data

In [19]:
filtered = iterative_drop(interactions_subset, 5, 5)
filtered

,user_id,track_id,play_count
2154,1,2154,4
2241,1,2241,4
3276,1,3270,6
4021,1,4003,2
5109,1,5080,2
...,...,...,...
519268882,120320,1480735,2
519269476,120320,2626622,3
519270322,120320,2465242,4
519272292,120320,14175074,2


In [20]:
filtered = filtered.merge(users)
filtered

,user_id,track_id,play_count,gender
0,1,2154,4,m
1,1,2241,4,m
2,1,3270,6,m
3,1,4003,2,m
4,1,5080,2,m
...,...,...,...,...
268362,120320,1480735,2,m
268363,120320,2626622,3,m
268364,120320,2465242,4,m
268365,120320,14175074,2,m


In [21]:
filtered.to_csv('sampled_interactions.tsv', sep='\t', index=False, header=False)